In [1]:
!pip install prometheus-api-client


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from prometheus_api_client import PrometheusConnect
from prometheus_api_client.metric_range_df import MetricRangeDataFrame
from datetime import datetime
from datetime import timedelta
from collections import defaultdict
import itertools


# 1. Настройка подключения
prom = PrometheusConnect(
    url="http://127.0.0.1:9090",  # замените на адрес вашего Prometheus
    disable_ssl=True               # отключаем проверку SSL, если нужно
)

# 2. Задаём параметры
query = 'histogram_quantile(0.9, sum(rate(common_event_delay_bucket[1m])) by (le))'
start_date = "2025-04-27 18:00:00"
end_date   = "2025-05-13 11:41:00"

start = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
end   = datetime.strptime(end_date,   "%Y-%m-%d %H:%M:%S")


def chunked_query_merge(prom, query, start, end, step="15s", chunk_days=1):
    chunk    = timedelta(days=chunk_days)
    curr     = start
    merged   = defaultdict(list)

    while curr < end:
        chunk_end = min(curr + chunk, end)

        for serie in prom.custom_query_range(query, curr, chunk_end, step=step):
            # tuple(sorted()) — устойчивый ключ по лейблам
            key = tuple(sorted(serie["metric"].items()))
            merged[key].extend(serie["values"])

        curr = chunk_end + timedelta(seconds=_parse_step_to_seconds(step))  # избегаем дубли

    # превращаем обратно в список объектов, как ждёт MetricRangeDataFrame
    return [
        {"metric": dict(key), "values": sorted(vals, key=lambda x: x[0])}
        for key, vals in merged.items()
    ]

raw = chunked_query_merge(prom, query, start, end, step="15s")
df  = MetricRangeDataFrame(raw)

# 5. Работа с данными
print(df.head())        # первые несколько строк
print(df.describe())    # базовая статистика

# 6. Сохранить в CSV, если нужно
df.to_csv("quantile_0.9_delay.csv", index=False)

NameError: name '_parse_step_to_seconds' is not defined

In [4]:
len(df)

89938